In [1]:
#! pip install timm

In [2]:
import pandas as pd
import numpy as np
import os


import cv2
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from torch.utils.data import Dataset, DataLoader

#from tqdm import tqdm

# tqdm doesn't work well in colab.
# This is the solution:
# https://stackoverflow.com/questions/41707229/tqdm-printing-to-newline
import tqdm.notebook as tq
#for i in tq.tqdm(...):


import gc

from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import StepLR


import albumentations as albu
from albumentations import Compose


from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, jaccard_score
import itertools


from skimage import measure


# load image with Pillow
from PIL import Image

from numpy import asarray

from skimage.transform import resize


import transformers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup, get_constant_schedule_with_warmup

#import segmentation_models_pytorch as smp
#from segmentation_models_pytorch.encoders import get_preprocessing_fn


import matplotlib.pyplot as plt
%matplotlib inline

# Don't Show Warning Messages
import warnings
warnings.filterwarnings('ignore')

# Note: Pytorch uses a channels-first format:
# [batch_size, num_channels, height, width]

print(torch.__version__)
print(torchvision.__version__)

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


2.5.1+cu121
0.20.1+cu121


In [3]:
# Set the seed values

import random

# Set the seed value all over the place to make this reproducible.
seed_val = 101

os.environ['PYTHONHASHSEED'] = str(seed_val)
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
torch.backends.cudnn.deterministic = True

In [4]:
base_path = '../input/forams-classification-2025'

notebook0 = 'exp24-forams-swin-exp4-w-patience-5'


model_0 = f'../input/{notebook0}/0_model.bin'
model_1 = f'../input/{notebook0}/1_model.bin'
model_2 = f'../input/{notebook0}/2_model.bin'
model_3 = f'../input/{notebook0}/3_model.bin'
model_4 = f'../input/{notebook0}/4_model.bin'

# image used: 'bottom'
MODEL_LIST1 = [model_0, model_1, model_2, model_3, model_4]

In [5]:
notebook1 = 'exp23-forams-swin-exp11-w-patience-5'


model_5 = f'../input/{notebook1}/0_model.bin'
model_6 = f'../input/{notebook1}/1_model.bin'
model_7 = f'../input/{notebook1}/2_model.bin'
model_8 = f'../input/{notebook1}/3_model.bin'
model_9 = f'../input/{notebook1}/4_model.bin'

# image used: 'top'
MODEL_LIST2 = [model_5, model_6, model_7, model_8, model_9]

In [6]:
import os

os.listdir('../input/')

['forams-classification-2025',
 'exp23-forams-swin-exp11-w-patience-5',
 'exp24-forams-swin-exp4-w-patience-5']

In [7]:
CLASS_LIST = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]

NUM_CLASSES = len(CLASS_LIST)

NUM_FOLDS = 5

START_FOLD = 0
STOP_FOLD = 5

NUM_EPOCHS = 30

PATIENCE = 5
IMAGE_SIZE = 384 # Swin requires size 384 images
NUM_CHANNELS = 3

L_RATE = 0.0001 # 0.00001
BATCH_SIZE = 8


NUM_CORES = os.cpu_count()
NUM_CORES

4

## Define the device

In [8]:
# For GPU

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

if torch.cuda.is_available():
    print('Num GPUs:', torch.cuda.device_count())
    print('GPU Type:', torch.cuda.get_device_name(0))

cuda:0
Num GPUs: 1
GPU Type: Tesla P100-PCIE-16GB


## Helper functions

In [9]:
# Check the timm model config to make sure this matches the model being used
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)

# The image must be of type uint8 or ToTensor() will not scale it correctly.
# assert image.dtype == 'uint8'

transform = transforms.Compose([transforms.ToTensor(),
                               normalize])


In [10]:
def display_images(df, remove_top_half=False):
    
    """
    Display jpg images in a grid.
    """
    
    plt.style.use("default")
    plt.figure(figsize=(13,12))

    rows = 1
    cols = 6

    for i in range(rows*cols):

        fname = df.loc[i, 'image_fname']
        label = df.loc[i, 'label']

        path = base_path + '/visualizations/visualizations/labelled/' + fname
        image = cv2.imread(path)

        if remove_top_half==True:
            # Remove the top half
            image = image[300:, :]  # Keep only the bottom 300 rows

        # Convert to grayscale
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Shape: (300, 300)

        ax = plt.subplot(rows, cols, i+1)
        plt.imshow(image)
        plt.title(label, color="black", fontsize=12)
        plt.axis("off")
    
    plt.tight_layout()
    plt.show()

In [11]:
def display_images_from_batch(data):
    
    """
    Display torch tensor images from the dataloader.
    """
    
    images = data['images']
    targets = data['targets']
    
    plt.style.use("default")
    plt.figure(figsize=(13,12))

    rows = 1
    cols = 6

    for i in range(rows*cols):

        image = images[i]
        label = targets[i]

        # convert from (C, H, W) to (H, W, C) format
        image = image.permute((1, 2, 0))

        # select the first channel only
        image = image[:, :, 0]

        ax = plt.subplot(rows, cols, i+1)
        plt.imshow(image)
        plt.title(label, color="black", fontsize=12)
        plt.axis("off")
    
    plt.tight_layout()
    plt.show()

## Create the dataloader

In [12]:
class CompDataset(Dataset):
    
    def __init__(self, df, augmentation=None, preprocessing=transform, source='train'):
        self.df = df
        self.augmentation = augmentation
        self.preprocessing = preprocessing
        self.source = source
        
    def __len__(self):
        return len(self.df)
        
        
        
    def __getitem__(self, index):
        
        # Load the image
        
        fname = self.df.loc[index, 'image_fname']
        image_used = self.df.loc[index, 'image_used']
        
        if self.source == 'train':
            #path = base_path + 'images/' + fname + '.jpg'
            path = base_path + '/visualizations/visualizations/labelled/' + fname
        else:
            #path = base_path + 'image_holdouts/' + fname + '.jpg'
            path = base_path + '/visualizations/visualizations/unlabelled/' + fname
        
        image = cv2.imread(path)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if image_used == 'bottom':
            # Remove the top half
            image = image[300:, :]
        else:
            # Remove the bottom half
            image = image[:300, :]

        # Convert to grayscale
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Convert back to BGR (3 channels but still looks gray)
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
        
        # resize the image
        # is cv2.INTER_CUBIC the same as bicubic interpolation?
        # bicubic is in the timm model config
        image = cv2.resize(image, 
                           (IMAGE_SIZE, IMAGE_SIZE), 
                           interpolation=cv2.INTER_CUBIC)
        
        
        # Target

        target = self.df.loc[index, 'label']
    
        target = torch.tensor(target, dtype=torch.long)
        

   
        # Apply augmentations
    
        if self.augmentation:
            
            sample = self.augmentation(image=image)
            image = sample['image']
            
        
        # Apply preprocessing
        
        if self.preprocessing:
            
            # The image must be of type uint8 or ToTensor() will not scale it correctly.
            assert image.dtype == 'uint8'
            
            image = self.preprocessing(image)
            
            
        return {
                "images": image,
                "targets": target
                }
    

## Engine - Training and Evaluation

In [13]:
# Define the inference function

def inference_fn(df, model, device, source='train'):
    
    data = CompDataset(df,
                       preprocessing=transform, source=source)

    data_loader = torch.utils.data.DataLoader(data,
                                            batch_size=BATCH_SIZE,
                                            shuffle=False,
                                           num_workers=NUM_CORES)
    
    model.eval()
    
    # This turns gradient calculations on and off.
    torch.set_grad_enabled(False)

    

    for i, data in tq.tqdm(enumerate(data_loader), total=len(data_loader)):
        
        images = data['images']

        images = images.to(device, dtype=torch.float)


        outputs = model(images)
        
        
        # convert to numpy so we can stack the batches.
        outputs = outputs.cpu().detach().numpy()
        
        # stack the preds from each batch
        if i == 0:
            fin_outputs = outputs
        else:
            fin_outputs = np.vstack((fin_outputs, outputs))
        
        

    return fin_outputs

## Define the model

In [14]:
import timm

model = timm.create_model('swin_large_patch4_window12_384', pretrained=True, 
                          num_classes=NUM_CLASSES)


model.safetensors:   0%|          | 0.00/801M [00:00<?, ?B/s]

## Make a test set prediction

## Load the test data

In [15]:
path = base_path + '/visualizations/visualizations/unlabelled'

image_list = os.listdir(path)

data = {
    "image_fname": image_list
}

df_test = pd.DataFrame(data)

df_test['label'] = 0

print(df_test.shape)

df_test.head()

(18216, 2)


,image_fname,label
0,foram_01254.jpg,0
1,foram_16347.jpg,0
2,foram_00700.jpg,0
3,foram_11030.jpg,0
4,foram_16839.jpg,0


In [16]:
# Add an id column

def get_id(x):
    item_id = x.split('_')[1]
    item_id = int(item_id.split('.')[0])

    return item_id

df_test['id'] = df_test['image_fname'].apply(get_id)

# Sort by 'id' in ascending order
df_test = df_test.sort_values(by='id', ascending=True)

# Reset the index
df_test = df_test.reset_index(drop=True)

# Reorder columns
new_order = ['id', 'image_fname', 'label']
df_test = df_test[new_order]

print(df_test.shape)

df_test.head()

(18216, 3)


,id,image_fname,label
0,0,foram_00000.jpg,0
1,1,foram_00001.jpg,0
2,2,foram_00002.jpg,0
3,3,foram_00003.jpg,0
4,4,foram_00004.jpg,0


## Make a prediction on bottom half of image

In [17]:
# Make a prediction on each fold model

df_test['image_used'] = 'bottom'

model_list = MODEL_LIST1

for i, path_model in enumerate(model_list):
    
    # lload the trained fold model
    model.load_state_dict(torch.load(path_model))
    model.to(device)
    
    test_preds = inference_fn(df_test, model, device, source='test')
    
    if i == 0:
        fin_preds = test_preds.copy()
        
    else:
        fin_preds = fin_preds + test_preds
        

# Average the preds
test_preds1 = fin_preds/len(model_list)


test_preds1.shape

  0%|          | 0/2277 [00:00<?, ?it/s]

  0%|          | 0/2277 [00:00<?, ?it/s]

  0%|          | 0/2277 [00:00<?, ?it/s]

  0%|          | 0/2277 [00:00<?, ?it/s]

  0%|          | 0/2277 [00:00<?, ?it/s]

(18216, 14)

## Make a prediction on top half of image

In [18]:
# Make a prediction on each fold model

df_test['image_used'] = 'top'

model_list = MODEL_LIST2

for i, path_model in enumerate(model_list):
    
    # lload the trained fold model
    model.load_state_dict(torch.load(path_model))
    model.to(device)
    
    test_preds = inference_fn(df_test, model, device, source='test')
    
    if i == 0:
        fin_preds = test_preds.copy()
        
    else:
        fin_preds = fin_preds + test_preds
        

# Average the preds
test_preds2 = fin_preds/len(model_list)


test_preds2.shape

  0%|          | 0/2277 [00:00<?, ?it/s]

  0%|          | 0/2277 [00:00<?, ?it/s]

  0%|          | 0/2277 [00:00<?, ?it/s]

  0%|          | 0/2277 [00:00<?, ?it/s]

  0%|          | 0/2277 [00:00<?, ?it/s]

(18216, 14)

## Ensemble the preds

In [19]:
# Average the preds
test_preds = (test_preds1 + test_preds2)/2.0

## 1. Create pseudo data¶

In [20]:
# Apply a softmax to the preds

# convert the val preds to a torch tensor
test_preds1 = torch.tensor(test_preds)

# CrossEntropyLoss has a softmax included.
softmax = nn.Softmax(dim=1)
test_preds1 = softmax(test_preds1)

# convert to numpy
test_preds1 = test_preds1.cpu().detach().numpy()

test_preds1.shape

(18216, 14)

In [21]:
# Get the max value in each row

test_preds_max = np.amax(test_preds1, axis=1)

test_preds_max.shape

(18216,)

In [22]:
# Get the index of the max value in each row

test_preds_int = np.argmax(test_preds1, axis=1)

test_preds_int.shape

(18216,)

In [23]:
# Put the preds into a dataframe

#df_test['name'] = pred_class_list
df_test['target'] = test_preds_int
df_test['score'] = test_preds_max

df_test['source'] = 'pseudo'


print(df_test.shape)

df_test.head()

(18216, 7)


,id,image_fname,label,image_used,target,score,source
0,0,foram_00000.jpg,0,top,10,0.999540,pseudo
1,1,foram_00001.jpg,0,top,1,0.661948,pseudo
2,2,foram_00002.jpg,0,top,2,0.760579,pseudo
3,3,foram_00003.jpg,0,top,4,0.542271,pseudo
4,4,foram_00004.jpg,0,top,10,0.999783,pseudo


In [24]:
len(df_test[df_test['score'] < 0.9])

11897

In [25]:
# Save as a csv file

path = 'df_pseudo.csv'
df_test.to_csv(path, index=False)

## 2. Ceate a submission

In [26]:
# Get the probabilities

# convert the test preds to a torch tensor
test_preds = torch.tensor(test_preds)

# take the softmax
softmax = nn.Softmax(dim=1)
test_preds = softmax(test_preds)

# convert to numpy
test_preds = test_preds.cpu().detach().numpy()

test_preds.shape

(18216, 14)

In [27]:
# take the argmax

test_preds = np.argmax(test_preds, axis=1)

test_preds.shape

(18216,)

In [28]:
# Convert to a list

test_preds = list(test_preds)

len(test_preds)

18216

## Create a submission csv file

In [29]:
path = base_path + '/sample_submission.csv'

df_sub = pd.read_csv(path)

print(df_sub.shape)

df_sub.head()

(18216, 2)


,id,label
0,0,9
1,1,2
2,2,3
3,3,9
4,4,11


## Add the preds to the sample submission

In [30]:
df_sub['label'] = test_preds

df_sub.head()

,id,label
0,0,10
1,1,1
2,2,2
3,3,4
4,4,10


In [31]:
# Create a submission csv file

path = 'submission.csv'
df_sub.to_csv(path, index=False)

In [32]:
# Create a requirements.txt file
# This is a list of all packages and their versions that were 
# used to create this solution.

!pip freeze > requirements.txt

In [33]:
!ls

df_pseudo.csv  __notebook__.ipynb  requirements.txt  submission.csv
